# ODRL Policy Validation

This validator uses the code from https://github.com/paolo7/ODRL-Engine.git

## Instructions

* Run the code block below.
* Select a function to perform from the dropdown menu
* Press the run button to make it run

Note: do not press run again until the current function is running. Wait for it to complete before running it again/selecting another one. If necessery re-run the code block to restart the application.

## Function Explanation

* `Upload ODRL File` use this function to upload an ODRL file, other functions will assume you have one uploaded.
* `File Info` display file name and size of the uploaded file
* `Visualise Policy` provides a basic visual way to explore an ODRL file by expanding its rules and components. Click on a comp
* `Full ODRL Validation` performs a diagnostic check of the ODRL file, checking for basic RDF file integrity and if the basic ODRL data structure is being followed (using SHACL), while providing aggregate statistics. This validator is in a beta stage, but it already covers the most common ODRL components.
* `Compare two graph for triple equality` this graph-diff utility function prompts the upload of two graphs, and checks for triple-by-triple equality, showing which triples appear in one graph but not the other.

### Contact
p.pareti@soton.ac.uk

In [2]:
!git clone https://github.com/paolo7/ODRL-Engine.git
%cd ODRL-Engine
from setup_colab import setup, show_interface
setup()
show_interface()

Dropdown(description='Select:', options=(('Upload ODRL File', 'upload'), ('File Info', 'fileinfo'), ('Visualis…

Button(button_style='success', description='Run', style=ButtonStyle())

Output()